**Prompt**: Implement Kruskal’s algorithm for finding the minimum spanning tree of a weighted graph. For
full credit, you must utilize a union-find data structure, as described in Section 4.6 of the Kleinberg/Tardos
textbook; i.e., as part of this assignment you must read and understand Section 4.6. Please note that
they prescribe several optimizations, and all of these should be present in your implementation as well.
You must also implement a mergesort algorithm.
You can only utilize the following primitive python data structures: python lists, python dictionaries, and
numpy arrays. You can only utilize primitive python and numpy functions, such as sum, multiplication,
division, exponentiation, etc. If you are in doubt, then please ask. Regardless of what operations and
data structures you use, you must conform to the API provided below.

Additional Software Specifications and I/O:
- All code must be written in Python.
- All numeric values should be floats, unless otherwise stated.
- You must use *exactly* the function names described below.
- Code must be delivered as a Python class, and include the methods specified below.
- You can assume graphs will have unique weights, and will be fully-connected.

obj = kruskalClass()
- Input:
- Output: an object instantiated from your class. Your class should have the name
‘kruskalClass’.

T = findMinimumSpanningTree(A)
- Input: ‘A’ is an NxN numpy array representing an adjacency matrix
representation of a graph. If 𝐴 it implies there is an edge between
𝑖𝑗 = 𝑤 > 0
nodes 𝑖 and 𝑗 with weight 𝑤. We will only consider undirected graphs, and
therefore ‘A’ will be an upper right triangular matrix. You can assume no two
edge weights are the same.
- Output: ‘T‘ is an NxN numpy array representing the minimum spanning tree of
‘A’. We will only consider undirected graphs, and therefore ‘T’ should be an
upper right triangular matrix.

b = mergesort(a)
- Input: ‘a’ must be a 1xK numpy array.
- Output: ‘b’ must be a 1xK numpy array with elements in ascending order (i.e.,
lowest value to highest).

u = makeUnionFind(N)
- Input: ‘N’ is the number of nodes in a graph.
- Output: ‘u’ is a 1XN python dictionary, where the keys are numerical labels for
the nodes, and the values are numpy arrays with the 0
th entry being a pointer to a
node in the same dictionary. The numpy arrays can be more than one 1-D if you
want, but they must be numpy arrays, and the 0
th entry must contain pointers to
other nodes that are used in the ‘find’ and ‘union’ functions

s = find(u,v)
- Input: ‘u’ is a union-find data structure. ‘v’ is a numerical index for a graph node.
- Output: ‘s’ is the numerical value corresponding to the label for the set of
connected nodes to which ‘v’ belongs.

u_out = union(u_in,s1,s2)
- Input: ‘u_in’ is a union-find data structure. ‘s1’ and ‘s2’ are numerical values
corresponding to the labels of two groups of graph nodes.
- Output: ‘u_out’ is the same as ‘u-in’, except that the two groups of nodes, ‘s1’
and ‘s2’, have been merged into a single group with a single label.

In [ ]:
class KruskalClass:

    def __init__(self):
        pass